## Importieren der Module

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path  
import glob

-----------

## Pfad einlesen

In [39]:
while True: 
    input_path = input("Was soll untersucht werden? allwords, nouns, adjectives oder verbs?").lower()
    
    if input_path == "allwords":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/all_words/clean_hitler"]
        break
    elif input_path == "nouns":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler"]
        break
    elif input_path == "verbs":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/verbs/clean_hitler"]
        break
    elif input_path == "adjectives":
        directory_path = ["/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/adjectives/clean_hitler"]
        break
    else: 
        print("Falsche Eingabe!")

text_files = []
for path in directory_path:
    text_files.extend(glob.glob(f"{path}/*.txt"))

print(len(text_files))
text_files[:10]

Was soll untersucht werden? allwords, nouns, adjectives oder verbs? nouns


29


['/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter15_buch02.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter03_buch01.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter03_buch02.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter11_buch01.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter01_buch01.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter07_buch02.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter14_buch02.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter09_buch01.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_hitler/chapter13_buch02.txt',
 '/home/simon/bachelorarbeit/vergleich_mk_trump/cleaned_docs/nouns/clean_

In [40]:
#directory_path =

In [41]:
#text_files = glob.glob(f"{directory_path}/*.txt")
#text_files[:10]

In [42]:
text_titles = [Path(text).stem for text in text_files]
print(len(text_titles))
text_titles[:10]

29


['chapter15_buch02',
 'chapter03_buch01',
 'chapter03_buch02',
 'chapter11_buch01',
 'chapter01_buch01',
 'chapter07_buch02',
 'chapter14_buch02',
 'chapter09_buch01',
 'chapter13_buch02',
 'chapter06_buch02']

--------------

## TF-IDF berechnen

In [43]:
tfidf_vectorizer = TfidfVectorizer(input='filename')

In [44]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

In [45]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [46]:
if input_path in ("allwords", "nouns"):
    tfidf_slice = tfidf_df[['immigration', 'illegal']]
    print(tfidf_slice.sort_index().round(decimals=2))
else:
    print("Keine Substantive enthalten!")

In [47]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,chapter15_buch02,abandonment,0.000000
1,chapter15_buch02,abbot,0.000000
2,chapter15_buch02,aberration,0.000000
3,chapter15_buch02,abettor,0.000000
4,chapter15_buch02,abeyance,0.000000
...,...,...,...
126029,chapter09_buch02,yield,0.000000
126030,chapter09_buch02,youngster,0.000000
126031,chapter09_buch02,youth,0.015259
126032,chapter09_buch02,zeppelin,0.000000


In [48]:
tfidf_df = tfidf_df.stack().reset_index()

In [49]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [50]:
tfidf_df.columns

Index(['document', 'term', 'tfidf'], dtype='object')

In [51]:
tfidf_df.sort_values(by=['tfidf','document'], ascending=[False,False]).groupby(['tfidf']).head(10)

,document,term,tfidf
85617,chapter06_buch01,propaganda,0.784629
24163,chapter07_buch02,meeting,0.747428
60491,chapter12_buch02,tradeunion,0.634141
47172,chapter10_buch02,state,0.553100
74420,chapter08_buch01,capital,0.529196
...,...,...,...
99963,epilogue_buch02,abhor,0.000000
99964,epilogue_buch02,ability,0.000000
99965,epilogue_buch02,abjuration,0.000000
99966,epilogue_buch02,abler,0.000000


In [52]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

In [53]:
top_tfidf[top_tfidf['term'].str.contains('propaganda')]

,document,term,tfidf
85617,chapter06_buch01,propaganda,0.784629
120385,chapter07_buch01,propaganda,0.180168
46503,chapter10_buch02,propaganda,0.108729
81271,chapter11_buch02,propaganda,0.323078


---------------

## TF-IDF visualisieren

In [54]:
import altair as alt
import numpy as np

#terme die gesondert mit rotem punkt ausgezeichnet werden
term_list = ['war', 'peace']

#randomness fürs ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

#visualisierungen mit ranking
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

#heatmap definieren
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

#roter punkt für oben festgelegte terme definieren
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

#text labels mit weiß für dunklere heatmap-einfärbungen
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

#visualisierungen darstellen
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)